In [3]:
import os
from kaggle_secrets import UserSecretsClient

try:
    GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
    os.environ["GOOGLE_GENAI_USE_VERTEXAI"] = "FALSE"
    print("✅ Gemini API key setup complete.")
except Exception as e:
    print(f"🔑 Authentication Error: Please make sure you have added 'GOOGLE_API_KEY' to your Kaggle secrets. Details: {e}")

✅ Gemini API key setup complete.


In [4]:
from google.adk.agents import Agent, SequentialAgent, ParallelAgent, LoopAgent
from google.adk.runners import InMemoryRunner
from google.adk.tools import AgentTool, FunctionTool, google_search
from google.genai import types

print("✅ ADK components imported successfully.")

✅ ADK components imported successfully.


In [9]:
# === 1) PARALLEL: Human-tone research in parallel ===
signals_researcher = Agent(
    name="SignalsResearcher",
    model="gemini-2.5-flash-lite",
    instruction=(
        "Research human-sounding writing signals: natural rhythm, voice consistency, mild imperfection, "
        "grounded specifics, sensory details, and emotional cadence. Provide short, citable notes."
    ),
    tools=[google_search],           # requires the import above
    output_key="signals_notes",
)
print("✅ signals_researcher created")

style_researcher = Agent(
    name="StyleResearcher",
    model="gemini-2.5-flash-lite",
    instruction=(
        "Collect practical techniques to avoid robotic tone: vary sentence length, prune fillers, "
        "prefer concrete verbs, show-not-tell. Provide actionable bullets."
    ),
    tools=[google_search],
    output_key="style_notes",
)
print("✅ style_researcher created")

examples_researcher = Agent(
    name="ExamplesResearcher",
    model="gemini-2.5-flash-lite",
    instruction=(
        "Find 2–3 short public examples (summarized) where AI text was mistaken as human or vice versa; "
        "extract what made it feel human. Keep it concise."
    ),
    tools=[google_search],
    output_key="examples_notes",
)
print("✅ examples_researcher created")

parallel_research = ParallelAgent(
    name="ParallelHumanToneResearch",
    sub_agents=[signals_researcher, style_researcher, examples_researcher],
)
print("✅ parallel_research created")


✅ signals_researcher created
✅ style_researcher created
✅ examples_researcher created
✅ parallel_research created


In [10]:
# === 2) AGGREGATE: ממזג את התוצרים מה-Parallel לתקציר עבודה אחד ===
aggregator = Agent(
    name="Aggregator",
    model="gemini-2.5-flash-lite",
    instruction=(
        "Merge signals_notes, style_notes, and examples_notes into a brief with: audience, angle, "
        "human-tone checklist (5–7 items), and 3 key claims. Return as a compact spec."
    ),
    output_key="human_tone_brief",
)
print("✅ Aggregator")

✅ Aggregator


In [12]:
# === 3) SEQUENTIAL: Outline -> Draft (בניית טיוטה על בסיס ה-brief) ===
outliner = Agent(
    name="Outliner",
    model="gemini-2.5-flash-lite",
    instruction=(
        "From human_tone_brief, produce an outline for a short post: Hook, 3 bullets, one reflective line, CTA, "
        "and hashtag themes (not final hashtags)."
    ),
    output_key="outline",
)
print("✅ Outliner")

writer = Agent(
    name="Writer",
    model="gemini-2.5-flash-lite",
    instruction=(
        "Write a 130–180 word post answering: 'Can AI write human-quality content?' "
        "Use the outline and the human-tone checklist implicitly. "
        "Rules: one-line hook (no emoji), 3 concrete bullets, one reflective sentence, end with 5–7 relevant hashtags."
    ),
    output_key="draft",
)
print("✅ Writer")


draft_pipeline = SequentialAgent(
    name="DraftPipeline",
    sub_agents=[outliner, writer],
)
print("✅ DraftPipeline")


✅ Outliner
✅ Writer
✅ DraftPipeline


In [14]:
# === 4) REFINEMENT LOOP: Critic -> Humanizer (שיפור איכות “אנושית”) ===
critic = Agent(
    name="Critic",
    model="gemini-2.5-flash-lite",
    instruction=(
        "Critique draft strictly by a human-tone rubric: natural flow, warmth, grounded specifics, rhythm variety, "
        "and clarity. Return: issues(list), fixes(list), and a score 0–10 (integer)."
    ),
    output_key="critique",
)
print("✅ Critic")


humanizer = Agent(
    name="Humanizer",
    model="gemini-2.5-flash-lite",
    instruction=(
        "Apply the critique fixes to produce an improved draft. "
        "Infuse light personal voice, mild imperfection (e.g., a short aside), and concrete detail—without fluff. "
        "Keep structure and length constraints."
    ),
    output_key="draft",  # overwrite
)
print("✅ Humanizer")

refinement_loop = LoopAgent(
    name="HumanToneRefinement",
    sub_agents=[critic, humanizer],
    max_iterations=2,  # שמרני, כמו בדוגמאות של הקורס
)
print("✅ HumanToneRefinement")


✅ Critic
✅ Humanizer
✅ HumanToneRefinement


In [15]:
# === 5) GUARD: בדיקות קומפליינס דקות לפני סיום ===
guard = Agent(
    name="Guard",
    model="gemini-2.5-flash-lite",
    instruction=(
        "Final pass: no over-promising, ≤3 emojis (not in hook), hashtags relevant, "
        "keep hook one line, preserve 130–180 words. "
        "Fix minimally if needed and return ONLY the final post."
    ),
    output_key="final_post",
)
print("✅ Guard")

✅ Guard


In [16]:
# === ROOT ORCHESTRATION: Parallel → Aggregate → Sequential → Loop → Guard ===
root = SequentialAgent(
    name="HumanLikePostPipeline",
    sub_agents=[parallel_research, aggregator, draft_pipeline, refinement_loop, guard],
)
print("✅ HumanLikePostPipeline")

runner = InMemoryRunner(agent=root)


✅ HumanLikePostPipeline


In [17]:
# === RUN: כאן שמים את ה-seed שלך (ה"פרומפט" ההתחלתי ל-root) ===
seed = """
Context: Day 1 of Google's 5-Day AI Agents Intensive (ADK).
Goal: Generate a professional yet human-sounding post for LinkedIn
that reflects my learning experience and thoughts from today.

Topic: Many content creators claim AI can't write human-quality content.
But after exploring ADK's agent orchestration and refinement loops today,
I started seeing how "human tone" itself can be an engineered outcome.

Audience: LinkedIn professionals in AI, product, or data fields — curious,
thoughtful people who enjoy insights and reflection more than hype.

Tone: warm, authentic, confident, learning-in-public.
Constraints: 130–180 words, short one-line hook, 3 main points or insights,
a reflective takeaway, and a few relevant hashtags at the end.
"""

response = await runner.run_debug(seed)
# הפוסט הסופי יופיע במפתח ה־state של 'final_post' (למשל בתצוגת ה-debug)



 ### Created new session: debug_session_id

User > 
Context: Day 1 of Google's 5-Day AI Agents Intensive (ADK).
Goal: Generate a professional yet human-sounding post for LinkedIn
that reflects my learning experience and thoughts from today.

Topic: Many content creators claim AI can't write human-quality content.
But after exploring ADK's agent orchestration and refinement loops today,
I started seeing how "human tone" itself can be an engineered outcome.

Audience: LinkedIn professionals in AI, product, or data fields — curious,
thoughtful people who enjoy insights and reflection more than hype.

Tone: warm, authentic, confident, learning-in-public.
Constraints: 130–180 words, short one-line hook, 3 main points or insights,
a reflective takeaway, and a few relevant hashtags at the end.

StyleResearcher > Here's a draft for your LinkedIn post, aiming for a warm, authentic, and reflective tone within your word count:

**AI's "Human Touch" is More Engineered Than We Think**

Day 1 of Go